In [54]:
import pandas as pd
import numpy as np
import time 
import requests as req
from bs4 import BeautifulSoup
from io import StringIO 

In [59]:
years= list(range(2025,2023,-1))

In [60]:
years

[2025, 2024]

In [61]:
all_matches=[]

In [68]:
url="https://fbref.com/en/comps/9/2023-2024-Premier-League.Stats"

In [74]:
for year in years:
    data=req.get(url)
    soup=BeautifulSoup(data.text)
    table=soup.select("table.stats_table")[0]
    links=table.find_all("a")
    links=[l.get("href") for l in links]
    links=[l for l in links if "/squads" in l]
    teams_urls=[f"https://fbref.com{l}" for l in links]

    previous_season=soup.select("a.prev")[0].get("href")
    standing_url=f"https://fbref.com/{previous_season}"

    for teams_url in teams_urls:
        team_name=teams_url.split("/")[-1].replace("-Stats","").replace("-"," ")
        
        data=req.get(teams_url)
        matches=pd.read_html(StringIO(data.text), match="Scores & Fixtures")

        soup=BeautifulSoup(data.text)
        links=soup.find_all("a")
        links=[l.get("href") for l in links]
        links=[l for l in links if l and 'all_comps/shooting/' in l]

        data= req.get(f"https://fbref.com{links[0]}")
        shooting=pd.read_html(data.text,match="Shooting")[0]
        shooting.columns=shooting.columns.droplevel()

        try:
            team_data=matches[0].merge(shooting[["Date","Sh","SoT","Dist","FK","PK","PKatt"]], on="Date")
        except ValueError:
            continue

        team_data=team_data[team_data["Comp"]=="Premier League"]
        team_data["Season"]=year
        team_data["Team"]=team_name
        all_matches.append(team_data)
        time.sleep(10)

C:\Users\USER\AppData\Local\Temp\ipykernel_4260\4289215898.py:25: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting=pd.read_html(data.text,match="Shooting")[0]
C:\Users\USER\AppData\Local\Temp\ipykernel_4260\4289215898.py:25: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting=pd.read_html(data.text,match="Shooting")[0]
C:\Users\USER\AppData\Local\Temp\ipykernel_4260\4289215898.py:25: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting=pd.read_html(data.text,match="Shooting")[0]
C:\Users\USER\AppData\Local\Temp\ipykernel_4260\4289215898.py:25: FutureWarning: Passing literal html to 'read_html' is deprecated

In [75]:
df=pd.concat(all_matches)

In [76]:
df

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt,Season,Team
0,2024-08-17,12:30,Premier League,Matchweek 1,Sat,Away,W,2.0,0.0,Ipswich Town,...,Match Report,NaN,18,5,14.8,0.0,0,0,2025,Liverpool
1,2024-08-25,16:30,Premier League,Matchweek 2,Sun,Home,W,2.0,0.0,Brentford,...,Match Report,NaN,19,8,13.6,1.0,0,0,2025,Liverpool
2,2024-09-01,16:00,Premier League,Matchweek 3,Sun,Away,W,3.0,0.0,Manchester Utd,...,Match Report,NaN,11,3,13.4,0.0,0,0,2025,Liverpool
3,2024-09-14,15:00,Premier League,Matchweek 4,Sat,Home,L,0.0,1.0,Nott'ham Forest,...,Match Report,NaN,14,5,14.9,0.0,0,0,2025,Liverpool
5,2024-09-21,15:00,Premier League,Matchweek 5,Sat,Home,W,3.0,0.0,Bournemouth,...,Match Report,NaN,19,12,16.6,0.0,0,0,2025,Liverpool
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,2024-08-25,14:00,Premier League,Matchweek 2,Sun,Home,L,2.0,6.0,Chelsea,...,Match Report,NaN,12,4,19.1,1.0,0,0,2024,Wolverhampton Wanderers
3,2024-08-31,15:00,Premier League,Matchweek 3,Sat,Away,D,1.0,1.0,Nott'ham Forest,...,Match Report,NaN,11,2,20.7,0.0,0,0,2024,Wolverhampton Wanderers
4,2024-09-15,16:30,Premier League,Matchweek 4,Sun,Home,L,1.0,2.0,Newcastle Utd,...,Match Report,NaN,12,5,15.2,0.0,0,0,2024,Wolverhampton Wanderers
6,2024-09-21,15:00,Premier League,Matchweek 5,Sat,Away,L,1.0,3.0,Aston Villa,...,Match Report,NaN,10,4,14.9,0.0,0,0,2024,Wolverhampton Wanderers


In [77]:
df.to_csv("matches_stats.csv")